<a href="https://colab.research.google.com/github/nlee100/bayesian-neuralnetworks/blob/main/model_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<table>
  <tr>
    <th>Title</th>
    <th>Model Exploration for PHP 2650 Final Project</th>
  </tr>
  <tr>
    <td>Creator</td>
    <td>Naomi Lee</td>
  </tr>
    <tr>
    <td>Content</td>
    <td>DNNSurv, BDNNSurv</td>
  </tr>
    <tr>
    <td>Github</td>
    <td><a href="https://github.com/nlee100/bayesian-neuralnetworks">bayesian-neuralnetworks</td>
  </tr>
  <tr>
    <td>Last Updated</td>
    <td>April 23, 2022</td>
  </tr>
</table>

---

<h2>DNNSurv</h2>

Deep neural network for survival analysis using pseudo values. Keras models in R. Attribution of work to L. Zhao and D. Feng.

In [2]:
# Neural network with two hidden layers.
pseudoDNN.train <- function(x_train, y_train){
  
  model <- keras_model_sequential() %>%
    layer_dense(units = 8, kernel_regularizer = regularizer_l2(0.0001), activation = "tanh",
                input_shape = dim(x_train)[[2]]) %>%
    layer_dense(units = 4, kernel_regularizer = regularizer_l2(0.01),
                activation = "tanh") %>%
    layer_dense(units = 1, activation='sigmoid')
  
  model %>% compile(
    optimizer = optimizer_adam(lr = 0.0025),
    loss = "mse",
    metrics = c("mae")
  )
  model %>% fit(x_train, y_train,
                epochs = 30, batch_size = 64,
                verbose = 0)
  
  model
}

In [21]:
# Neural network with one hidden layer.
pseudoDNN.train <- function(x_train, y_train){
  # Use selu instead of relu for some studies...#TODO: Explore.
  model <- keras_model_sequential() %>%
    layer_dense(units=16,  activation = "selu",bias_initializer = initializer_constant(0.0),
                input_shape = dim(x_train)[[2]]) %>%
    layer_dropout(rate = 0.2) %>%
    layer_dense(units = 1, activation='sigmoid')
  
  model %>% compile(
    optimizer = optimizer_rmsprop(lr = 0.001),
    loss = "mse",
    metrics = c("mae")
  )
  model %>% fit(x_train, y_train,
                epochs = 1000, batch_size =256,
                verbose = 0)
  model
}

In [22]:
# Prediction.
pseudoDNN.predict <- function(model, x_test){
  ypred <- model %>% predict(x_test)
  ypred
}

In [23]:
# Get pseudo conditional survival probabilities, where
# t = the survival time, d = the censoring indicator, and
# qt = a vector of time points that are used to divide the time interval.
getPseudoConditional <- function(t, d, qt){
  #browser()
  s <- c(0, qt)  
  n=length(t)
  ns=length(s)-1  # the number of intervals
  D <- do.call(cbind, lapply(1:ns, function(j)  (s[j] < t) * (t <= s[j+1]) * (d == 1)))
  R <- do.call(cbind, lapply(1:ns, function(j) ifelse(s[j] < t, 1, 0)))
  Delta<-do.call(cbind, lapply(1:ns, function(j) pmin(t,s[j+1])-s[j]))
  
  # Format into long formate.
  dd.tmp=cbind.data.frame(id=rep(1:n,ns),s=rep(c(0,qt[-length(qt)]), each=n), y=c(R*Delta),d=c(D))
  
  dd=dd.tmp[dd.tmp$y>0,]
  pseudost=rep(NA, nrow(dd))
  for (j in 1:ns){
    index= (dd$s==s[j])
    dds=dd[index,]
    pseudost[index]=pseudosurv(time=dds$y, event=dds$d, tmax=s[j+1]-s[j])$pseudo
    print(j)
  }
  dd$pseudost=pseudost  
  
  return(dd[,c(1,2,5)])
}

In [18]:
rm(list=ls())
# install.packages(c("keras","pseudo","survivalROC", "survival", "survAUC"))

# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("survcomp")

library(keras)
library(pseudo)
library(survivalROC)
library(survival)
library(survcomp)
library(survAUC)

Loading required package: prodlim



In [24]:
# Simulate survival data from a cox model with one covariate. 
set.seed(2412)
pickTime=c(0.7,1.7,3.2,5.3,8.3)
n=2000
x=rnorm(n,0,1)
c0=0.1
times=rexp(n,c0*exp(1*x))  
time.censor=rexp(n,c0*exp(1*x)) 
summary(time.censor)
delta=ifelse(times<time.censor, 1, 0)
time <- ifelse(times<time.censor, times, time.censor)

x_train  <- x[1:(n/2)]
surv_train <- time[1:(n/2)]
cen_train <- delta[1:(n/2)]

# Data normalization.
mean <- apply(as.matrix(x_train), 2, mean)
std <- apply(as.matrix(x_train), 2, sd)
x_train <- scale(x_train, center = mean, scale = std)

# Data normalization.
x_test=x[(n/2+1):n]
surv_test <- time[(n/2+1):n]
cen_test =delta[(n/2+1):n]
x_test<- scale(x_test, center = mean, scale = std)

# Get the pseudo conditinal survival probability. 
pseudoCond  = getPseudoConditional(surv_train, cen_train, pickTime)

# Covariate.
x <- x_train[pseudoCond$id,]

# Create dummy variables for the time points.
smatrix=model.matrix(~as.factor(pseudoCond$s)+0)

# Create input predictors. 
x_train.all <- cbind(x, smatrix)

y_train.all <- pseudoCond$pseudost

model = pseudoDNN.train(x_train.all, y_train.all)


# Format the test data.
x_test.all=do.call(rbind, replicate(length(pickTime), x_test, simplify=FALSE))
s_test=rep(pickTime,each=nrow(x_test))
smatrix.test=model.matrix(~as.factor(s_test)+0)
x_test.all=cbind(x_test.all,smatrix.test)

# Predict test data.
ypred.con <- pseudoDNN.predict(model, x_test.all)

# Obtain the marginal survival probability by multiple series of conditional probabilities.
ypred.con <- matrix(ypred.con, nrow=nrow(x_test))
ypred <- lapply(1:length(c(0, qt)), function(i) apply(ypred.con[,1:i, drop=FALSE], 1, prod))
surv_prob <- Reduce(cbind, ypred)

# c-index at time 0.7.
concordance.index(x=1-surv_prob[,2], surv.time=surv_test, surv.event=cen_test, method="noether")$c.index

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
  0.0019   2.3034   6.5175  16.9276  17.3017 529.2871 

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5


Warning message in backcompat_fix_rename_lr_to_learning_rate(...):
“the `lr` argument has been renamed to `learning_rate`.”


[1] 0.742092

<h2>BDNNSurv</h2>

Bayesian heirchical deep neural network for survival analysis using pseudo values. Data simulation in R. Attribution of work to D. Feng and L. Zhao .

In [5]:
### Obtain true values by simulation.
a=1.1; b=0.8
beta2=0.1

n <- 10000000
set.seed(100)
u <- runif(n, 0, 1)
z <- rbinom(n, 1, 0.5)
nvars <- 10
XX <- matrix(rnorm(nvars*n), nrow=n, ncol=nvars)
rho <- 0.5    
V <- matrix(c(1, rho, rho^2,
              rho, 1, rho^2,
              rho^2, rho, 1), 3, 3)
qf <- rep(0, n)
for(i in 1:n){
    qf[i] <- t(XX[i,1:3]) %*% V %*% XX[i,1:3]
}    


y1 <- as.vector(b*(exp(-log(u)*exp(-qf*beta2))-1)^(1/a)) 
print(quantile(y1, seq(0.1, 0.9, by=0.1)))
#       10%        20%        30%        40%        50%        60%        70%        80%        90% 
#0.07864246 0.16315466 0.26375659 0.38902902 0.55357536 0.78438583 1.13899336 1.78226636 3.44708184 

### Simulate data.

rm(list=ls())

install.packages(c("survival", "pseudo"))

library(survival)
library(pseudo)

#obtain pseudo-values for a data set
getPseudo <- function(time, event, tmax){
    
    pseudodt <- 1 - pseudosurv(time=time, event=event,
                               tmax=tmax)$pseudo
    
    pseudodt
}

#TODO: Save data locally and split into test/train for network.

       10%        20%        30%        40%        50%        60%        70% 
0.07852361 0.16285140 0.26326659 0.38861972 0.55317008 0.78385029 1.13883537 
       80%        90% 
1.78046938 3.44271251 


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘KMsurv’, ‘geepack’


Loading required package: KMsurv

Loading required package: geepack



# Relevant Work

<ul type='disc'>
  <li>RNN-SURV</li>
  <li>DeepSurv</li>
</ul> 